## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config3 import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [28]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/anuoluwaposoniyi/Desktop/Class/Weather_Database/Output/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lon,Max Temp,Humidity,Cloudiness,Current Description
0,0,busselton,AU,-33.6500,115.3333,57.20,66,85,overcast clouds
1,1,sampit,ID,-2.5333,112.9500,75.47,90,90,overcast clouds
2,2,bluff,NZ,-46.6000,168.3333,46.62,85,6,clear sky
3,3,ushuaia,AR,-54.8000,-68.3000,51.46,71,75,broken clouds
4,4,cherskiy,RU,68.7500,161.3000,6.48,94,100,overcast clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the preferred minimum temperature for your trip?"))
max_temp = float(input("What is the preferred maximum temperature for your trip?"))


What is the preferred minimum temperature for your trip?71
What is the preferred maximum temperature for your trip?87


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df

,Unnamed: 0,City,Country,Lat,Lon,Max Temp,Humidity,Cloudiness,Current Description
1,1,sampit,ID,-2.5333,112.9500,75.47,90,90,overcast clouds
7,7,ambunti,PG,-4.2365,142.8448,74.08,99,100,overcast clouds
13,13,kalmunai,LK,7.4167,81.8167,79.12,86,56,broken clouds
14,14,najran,SA,17.4924,44.1277,78.64,26,98,overcast clouds
18,18,cayenne,GF,4.9333,-52.3333,84.24,6,75,broken clouds
...,...,...,...,...,...,...,...,...,...
680,680,merauke,ID,-8.4667,140.3333,79.07,85,59,broken clouds
682,682,dipalpur,PK,30.6709,73.6529,85.12,12,11,few clouds
686,686,nirmali,IN,26.3167,86.5833,78.58,65,0,clear sky
688,688,gat,IL,31.6100,34.7642,86.68,24,8,clear sky


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                1
Lat                    0
Lon                    0
Max Temp               0
Humidity               0
Cloudiness             0
Current Description    0
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,Unnamed: 0,City,Country,Lat,Lon,Max Temp,Humidity,Cloudiness,Current Description
1,1,sampit,ID,-2.5333,112.9500,75.47,90,90,overcast clouds
7,7,ambunti,PG,-4.2365,142.8448,74.08,99,100,overcast clouds
13,13,kalmunai,LK,7.4167,81.8167,79.12,86,56,broken clouds
14,14,najran,SA,17.4924,44.1277,78.64,26,98,overcast clouds
18,18,cayenne,GF,4.9333,-52.3333,84.24,6,75,broken clouds
...,...,...,...,...,...,...,...,...,...
680,680,merauke,ID,-8.4667,140.3333,79.07,85,59,broken clouds
682,682,dipalpur,PK,30.6709,73.6529,85.12,12,11,few clouds
686,686,nirmali,IN,26.3167,86.5833,78.58,65,0,clear sky
688,688,gat,IL,31.6100,34.7642,86.68,24,8,clear sky


In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lon"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lon,Hotel Name
1,sampit,ID,75.47,overcast clouds,-2.5333,112.9500,
7,ambunti,PG,74.08,overcast clouds,-4.2365,142.8448,
13,kalmunai,LK,79.12,broken clouds,7.4167,81.8167,
14,najran,SA,78.64,overcast clouds,17.4924,44.1277,
18,cayenne,GF,84.24,broken clouds,4.9333,-52.3333,
19,victoria,HK,72.18,overcast clouds,22.2855,114.1577,
24,faanui,PF,81.50,few clouds,-16.4833,-151.7500,
27,tarauaca,BR,83.50,scattered clouds,-8.1614,-70.7656,
29,avarua,CK,80.65,broken clouds,-21.2078,-159.7750,
32,iralaya,HN,84.13,clear sky,15.0000,-83.2333,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lon = row["Lon"]
    params["location"]=f"{lat},{lon}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found. Skipping...")
        

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [29]:
# 7. Drop the rows where there is no Hotel Name.
x = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = x.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lon,Hotel Name
1,sampit,ID,75.47,overcast clouds,-2.5333,112.9500,Aquarius Boutique Hotel Sampit
7,ambunti,PG,74.08,overcast clouds,-4.2365,142.8448,AMBUNTI HIDE INN
13,kalmunai,LK,79.12,broken clouds,7.4167,81.8167,VS Villa
14,najran,SA,78.64,overcast clouds,17.4924,44.1277,محمد هادي ال فطيح
18,cayenne,GF,84.24,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
...,...,...,...,...,...,...,...
677,pokhara,NP,72.90,clear sky,28.2333,83.9833,OYO 170 Hotel View Point
678,manggar,ID,81.00,broken clouds,-2.8833,108.2667,Guest Hotel
682,dipalpur,PK,85.12,few clouds,30.6709,73.6529,Kalay Khan Nashta کالے خان ناشتہ
686,nirmali,IN,78.58,clear sky,26.3167,86.5833,Baidyanath Sharma Lodge


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/anuoluwaposoniyi/Desktop/Class/Weather_Database/Output/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotelinfo = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lon"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
markerlayer = gmaps.marker_layer(locations, info_box_content=hotelinfo)

# 11b. Display the figure
fig = gmaps.figure()
fig.add_layer(markerlayer)



In [25]:
# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))